In [3]:
import sys
from pathlib import Path
import pandas as pd

sys.path.insert(0, str(Path.cwd().parent))

import numpy as np
from cogwheel import data, gw_utils, gw_plotting, utils
from dot_pe import inference


# Generate synthetic data for inference

In [5]:
# create data
event_data_kwargs = {
    "detector_names": "HLV",
    "duration": 120.0,
    "asd_funcs": ["asd_H_O3", "asd_L_O3", "asd_V_O3"],
    "tgps": 0.0,
    "fmax": 1600.0,
}


event_data = data.EventData.gaussian_noise(
    eventname="example_eventdata", **event_data_kwargs, seed=20250311
)

mchirp = 75
q = 1 / 2

m1, m2 = gw_utils.mchirpeta_to_m1m2(mchirp, gw_utils.q_to_eta(q))
injection_par_dic = dict(
    m1=m1,
    m2=m2,
    ra=0,
    dec=0,
    iota=np.pi * 7 / 16,
    psi=1.0,
    phi_ref=12.0,
    s1z=0.6,
    s2z=0.6,
    s1x_n=0.1,
    s1y_n=0.2,
    s2x_n=0.3,
    s2y_n=-0.2,
    l1=0.0,
    l2=0.0,
    tgps=0.0,
    f_ref=50.0,
    d_luminosity=1e3,
    t_geocenter=0.0,
)

event_data.inject_signal(injection_par_dic, "IMRPhenomXODE")

print(event_data.injection["d_h"] - event_data.injection["h_h"] / 2)
print(sum(event_data.injection["d_h"] - event_data.injection["h_h"] / 2))

[189.58784798 389.95858028  26.19967493]
605.7461031798991


In [ ]:
# Alternatively, load data form file
event_data = data.EventData.from_npz(filename="example_eventdata.npz")

Set bank folder, in the relevant mass range.

See how to create mass ranges in create_sample_bank.ipynb


In [6]:
bank_folder = "/Users/jonatahm/Work/GW/magic_integral/banks/bank_mchirp_50"
bank_folder = Path(bank_folder)
event_dir = event_data.eventname

In [ ]:
rundir = inference.run(
    event_dir=event_dir,
    event=event_data,
    bank_folder=bank_folder,
    n_int=2**14,
    n_ext=128,
    n_phi=32,
    n_t=64,
    i_int_start=0,
    blocksize=2**10,
    single_detector_blocksize=2**10,
    seed=42,
    size_limit=10**6,
    draw_subset=True,
    n_draws=None,
)

TypeError: run() got an unexpected keyword argument 'use_reps'

In [ ]:
rundirs = sorted(
    Path("example_eventdata").glob("run_*"),
    key=lambda x: int(x.name.split("_")[-1]),
)

rundir = rundirs[-1]
summary_results = utils.read_json(rundirs[-1] / "summary_results.json")

for k, v in summary_results.items():
    print(k, v)
    

In [ ]:
samples = pd.read_feather(rundirs[-1] / "samples.feather")
params = ["mchirp", "lnq", "chieff", "cumchidiff", "costheta_jn", "lnl"]
gw_plotting.CornerPlot(samples, params=params, smooth=1, tail_probability=1e-2).plot()